# EXP_001 — Restriction Site Analysis for Golden Gate Genome Tiling

**Goal:** Pick the best Type IIS restriction enzyme for Golden Gate assembly of ~7 kb PCR tiles covering the *E. coli* MG1655 genome.

**Pipeline:** 7 kb PCR fragments → Lvl0 MoClo plasmids → 11-fragment Golden Gate → Lvl1 T7 replisome plasmids

**Key question:** Which enzyme has the fewest recognition sites in the genome, so we can eliminate internal sites during PCR?

In [ ]:
# ── Imports & config ──────────────────────────────────────────────
import sys, os
sys.path.insert(0, '.')

from restriction_utils import (
    download_mg1655, find_all_sites, site_stats,
    sites_per_window, site_free_windows, TYPE_IIS_ENZYMES,
)
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

# ── Dark theme ─────────────────────────────────────────────────────
DARK_BG   = '#0d1117'
CARD_BG   = '#161b22'
GRID_CLR  = '#21262d'
TEXT_CLR  = '#c9d1d9'
ACCENT    = '#58a6ff'
GREEN     = '#3fb950'
YELLOW    = '#d29922'
ORANGE    = '#db6d28'
RED       = '#f85149'
PURPLE    = '#bc8cff'
PALETTE   = ['#58a6ff', '#3fb950', '#d29922', '#db6d28', '#f85149', '#bc8cff', '#f778ba', '#79c0ff']

LAYOUT_DEFAULTS = dict(
    template='plotly_dark',
    paper_bgcolor=DARK_BG,
    plot_bgcolor=CARD_BG,
    font=dict(family='Inter, system-ui, sans-serif', size=13, color=TEXT_CLR),
    margin=dict(l=60, r=30, t=60, b=50),
)

AXIS_DEFAULTS = dict(
    gridcolor=GRID_CLR,
    zerolinecolor=GRID_CLR,
)

os.makedirs('data', exist_ok=True)

def save_fig(fig, name, width=1400, height=600):
    """Save figure as both interactive HTML and static PNG."""
    fig.write_html(f'data/{name}.html', include_plotlyjs='cdn')
    fig.write_image(f'data/{name}.png', width=width, height=height, scale=2)
    print(f'  ✓ data/{name}.html + data/{name}.png')

---
## 1. Download & Load the E. coli MG1655 Genome

In [ ]:
record = download_mg1655()
genome_len = len(record.seq)

n_genes = sum(1 for f in record.features if f.type == 'gene')
n_cds   = sum(1 for f in record.features if f.type == 'CDS')

print(f'Organism:  {record.annotations["organism"]}')
print(f'Accession: {record.id}')
print(f'Length:    {genome_len:,} bp')
print(f'Genes:     {n_genes:,}')
print(f'CDS:       {n_cds:,}')

---
## 2. Map All Type IIS Restriction Sites

In [ ]:
all_sites = find_all_sites(record)

rows = []
for name in sorted(all_sites, key=lambda n: len(all_sites[n])):
    pos = all_sites[name]
    stats = site_stats(pos, genome_len)
    free = site_free_windows(pos, genome_len, window_size=7000)
    stats['enzyme'] = name
    enz = TYPE_IIS_ENZYMES[name]
    stats['recognition'] = str(enz.site)
    stats['site_free_7kb_windows'] = len(free)
    total_free_bp = sum(length for _, length in free)
    stats['pct_genome_free_7kb'] = round(100 * total_free_bp / genome_len, 1)
    rows.append(stats)

df = pd.DataFrame(rows).set_index('enzyme')
df_display = df[['recognition', 'count', 'density_per_kb', 'mean_spacing',
                  'min_spacing', 'max_spacing', 'site_free_7kb_windows',
                  'pct_genome_free_7kb']].copy()
df_display.columns = ['Recognition', 'Sites', 'Sites/kb', 'Mean gap (bp)',
                       'Min gap (bp)', 'Max gap (bp)', 'Free 7kb windows',
                       '% genome in free 7kb+']
df_display = df_display.round(2)
df_display

---
## 3. Genome-Wide Visualisation

### 3a. Site positions along the genome

In [ ]:
enzymes_sorted = sorted(all_sites, key=lambda n: len(all_sites[n]))

fig = make_subplots(
    rows=len(enzymes_sorted), cols=1,
    shared_xaxes=True,
    vertical_spacing=0.02,
    subplot_titles=[f'{n} ({len(all_sites[n])} sites)' for n in enzymes_sorted],
)

for i, (name, color) in enumerate(zip(enzymes_sorted, PALETTE)):
    pos = all_sites[name]
    fig.add_trace(
        go.Scatter(
            x=np.repeat([p / 1e6 for p in pos], 3),
            y=np.tile([0, 1, None], len(pos)),
            mode='lines',
            line=dict(color=color, width=0.8),
            name=name,
            hovertemplate='%{x:.3f} Mb<extra>' + name + '</extra>',
            showlegend=False,
        ),
        row=i+1, col=1,
    )
    fig.update_yaxes(showticklabels=False, showgrid=False, row=i+1, col=1, **AXIS_DEFAULTS)
    fig.update_xaxes(showgrid=False, row=i+1, col=1, **AXIS_DEFAULTS)

fig.update_xaxes(title_text='Genome position (Mb)', row=len(enzymes_sorted), col=1)
fig.update_layout(
    title='Type IIS restriction sites across E. coli MG1655',
    height=120 * len(enzymes_sorted),
    **LAYOUT_DEFAULTS,
)
for ann in fig.layout.annotations:
    ann.font.color = TEXT_CLR
    ann.font.size = 11

save_fig(fig, 'site_positions_linear', height=120 * len(enzymes_sorted))
fig.show()

### 3b. Site density in 7 kb sliding windows (top 3 candidates)

In [ ]:
top_enzymes = ['BsaI', 'SapI', 'AarI']

fig = make_subplots(
    rows=len(top_enzymes), cols=1,
    shared_xaxes=True,
    vertical_spacing=0.08,
    subplot_titles=[f'{n}' for n in top_enzymes],
)

for i, name in enumerate(top_enzymes):
    window_data = sites_per_window(all_sites[name], genome_len, window_size=7000)
    positions_w = [d[0] / 1e6 for d in window_data]
    counts_w    = [d[1] for d in window_data]
    
    colors_w = [GREEN if c == 0 else YELLOW if c == 1
                else ORANGE if c == 2 else RED for c in counts_w]
    
    n_free = sum(1 for c in counts_w if c == 0)
    pct_free = 100 * n_free / len(counts_w)
    
    fig.add_trace(
        go.Bar(
            x=positions_w,
            y=counts_w,
            marker_color=colors_w,
            name=name,
            showlegend=False,
            hovertemplate='%{x:.2f} Mb: %{y} sites<extra>' + name + '</extra>',
        ),
        row=i+1, col=1,
    )
    fig.update_yaxes(title_text='sites in 7kb', row=i+1, col=1, **AXIS_DEFAULTS)
    fig.update_xaxes(showgrid=False, row=i+1, col=1, **AXIS_DEFAULTS)

fig.update_xaxes(title_text='Genome position (Mb)', row=len(top_enzymes), col=1)
fig.update_layout(
    title='Sites per 7 kb sliding window — top 3 candidates',
    height=250 * len(top_enzymes),
    bargap=0,
    **LAYOUT_DEFAULTS,
)
for ann in fig.layout.annotations:
    ann.font.color = TEXT_CLR
    ann.font.size = 11

save_fig(fig, 'site_density_7kb_window', height=250 * len(top_enzymes))
fig.show()

### 3c. Gap-size distribution between consecutive sites

In [ ]:
fig = make_subplots(rows=1, cols=3, subplot_titles=top_enzymes, horizontal_spacing=0.08)

for i, name in enumerate(top_enzymes):
    pos = sorted(all_sites[name])
    gaps = [pos[j+1] - pos[j] for j in range(len(pos)-1)]
    gaps.append(genome_len - pos[-1] + pos[0])  # circular wrap
    
    n_above = sum(1 for g in gaps if g >= 7000)
    col_idx = '' if i == 0 else str(i + 1)
    
    fig.add_trace(
        go.Histogram(
            x=gaps, nbinsx=80,
            marker_color=PALETTE[i],
            marker_line=dict(color=DARK_BG, width=0.5),
            name=name,
            showlegend=False,
            hovertemplate='Gap: %{x:,.0f} bp<br>Count: %{y}<extra></extra>',
        ),
        row=1, col=i+1,
    )
    fig.add_vline(
        x=7000, line=dict(color=RED, width=2, dash='dash'),
        row=1, col=i+1,
    )
    fig.add_annotation(
        text=f'{n_above} gaps ≥ 7 kb',
        xref=f'x{col_idx} domain', yref=f'y{col_idx} domain',
        x=0.95, y=0.95, xanchor='right', yanchor='top',
        showarrow=False,
        font=dict(size=11, color=TEXT_CLR),
        bgcolor=CARD_BG, borderpad=4,
        bordercolor=GRID_CLR, borderwidth=1,
    )
    fig.update_xaxes(title_text='Gap (bp)', row=1, col=i+1, **AXIS_DEFAULTS)
    fig.update_yaxes(title_text='Count' if i == 0 else '', row=1, col=i+1, **AXIS_DEFAULTS)

fig.update_layout(
    title='Gap size distribution (distance between consecutive sites)',
    height=450,
    **LAYOUT_DEFAULTS,
)
for ann in fig.layout.annotations:
    if ann.font.color is None:
        ann.font.color = TEXT_CLR

save_fig(fig, 'gap_distribution', height=450)
fig.show()

---
## 4. Enzyme Ranking

In [ ]:
enzyme_names = df_display.index.tolist()
bar_colors = [GREEN if n == enzyme_names[0] else '#484f58' for n in enzyme_names]

fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=['Total sites in genome', 'Largest site-free stretch', '% genome in free ≥ 7 kb stretches'],
    horizontal_spacing=0.12,
)

fig.add_trace(go.Bar(
    y=enzyme_names, x=df_display['Sites'].values,
    orientation='h', marker_color=bar_colors,
    hovertemplate='%{y}: %{x} sites<extra></extra>',
    showlegend=False,
), row=1, col=1)

fig.add_trace(go.Bar(
    y=enzyme_names, x=df_display['Max gap (bp)'].values,
    orientation='h', marker_color=bar_colors,
    hovertemplate='%{y}: %{x:,.0f} bp<extra></extra>',
    showlegend=False,
), row=1, col=2)
fig.add_vline(x=7000, line=dict(color=RED, width=2, dash='dash'), row=1, col=2)
fig.add_annotation(
    text='7 kb', x=7000, y=len(enzyme_names)-0.5,
    xref='x2', yref='y2',
    showarrow=False, font=dict(size=10, color=RED),
)

fig.add_trace(go.Bar(
    y=enzyme_names, x=df_display['% genome in free 7kb+'].values,
    orientation='h', marker_color=bar_colors,
    hovertemplate='%{y}: %{x:.1f}%<extra></extra>',
    showlegend=False,
), row=1, col=3)

fig.update_yaxes(autorange='reversed', **AXIS_DEFAULTS)
fig.update_xaxes(**AXIS_DEFAULTS)

fig.update_layout(
    title='Type IIS enzyme comparison for Golden Gate genome tiling',
    height=500, width=1400,
    **LAYOUT_DEFAULTS,
)
for ann in fig.layout.annotations:
    if hasattr(ann.font, 'color') and (ann.font.color is None or ann.font.color == TEXT_CLR):
        ann.font.color = TEXT_CLR
        ann.font.size = 12

save_fig(fig, 'enzyme_ranking', height=500)
fig.show()

---
## 5. Best Enzyme — Detailed Breakdown

In [ ]:
best = enzyme_names[0]
best_pos = sorted(all_sites[best])
best_gaps = [best_pos[i+1] - best_pos[i] for i in range(len(best_pos)-1)]
best_gaps.append(genome_len - best_pos[-1] + best_pos[0])

n_tiles_site_free = sum(1 for g in best_gaps if g >= 7000)
bp_site_free = sum(g for g in best_gaps if g >= 7000)

print(f'Best enzyme: {best}')
print(f'Recognition: {TYPE_IIS_ENZYMES[best].site}')
print(f'Total sites: {len(best_pos)}')
print()
print(f'Site-free stretches >= 7 kb:  {n_tiles_site_free}')
print(f'  -> covering {bp_site_free:,} bp ({100*bp_site_free/genome_len:.1f}% of genome)')
print()

window_data = sites_per_window(best_pos, genome_len, window_size=7000)
site_counts = [d[1] for d in window_data]
print('Tiles needing domestication (sites within a 7kb window):')
labels, values, clrs = [], [], []
color_map = {0: GREEN, 1: YELLOW, 2: ORANGE}
for ns in range(max(site_counts) + 1):
    nw = sum(1 for c in site_counts if c == ns)
    pct = 100 * nw / len(site_counts)
    bar = chr(9608) * int(pct / 2)
    print(f'  {ns} sites -> {nw:4d} windows ({pct:5.1f}%)  {bar}')
    labels.append(f'{ns} site{"s" if ns != 1 else ""}')
    values.append(nw)
    clrs.append(color_map.get(ns, RED))

fig = go.Figure(go.Pie(
    labels=labels, values=values,
    hole=0.55,
    marker=dict(colors=clrs, line=dict(color=DARK_BG, width=2)),
    textinfo='label+percent',
    textfont=dict(size=13, color=TEXT_CLR),
    hovertemplate='%{label}: %{value} windows (%{percent})<extra></extra>',
))
fig.update_layout(
    title=f'{best} sites per 7 kb tile — domestication effort',
    height=500, width=700,
    **LAYOUT_DEFAULTS,
    annotations=[dict(
        text=f'{best}<br>{len(best_pos)} sites',
        x=0.5, y=0.5, font_size=16, font_color=TEXT_CLR,
        showarrow=False,
    )],
)

save_fig(fig, 'domestication_donut', width=700, height=500)
fig.show()

In [ ]:
df_display.to_csv('data/restriction_site_summary.csv')
print('Done. Summary saved to data/restriction_site_summary.csv')